In [2]:
# Import retro to play Street Fighter using a ROM
import retro
# Import time to slow down game
import time

In [15]:
# %%python -m retro.import . # Run this from the roms folder, or where you have your game roms 

UsageError: %%python is a cell magic, but the cell body is empty.


In [3]:
# Import environment base class for a wrapper 
from gym import Env 
# Import the space shapes for the environment
from gym.spaces import MultiBinary, Box
# Import numpy to calculate frame delta 
import numpy as np
# Import opencv for grayscaling
import cv2
# Import matplotlib for plotting the image
from matplotlib import pyplot as plt

In [75]:
# Create custom environment 
class StreetFighter(Env): # pass in basic env from above to preprocessing
    def __init__(self):
        super().__init__() # inherit from base env
        # Specify action space and observation space 
        self.observation_space = Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8) # grayscaled frame, smaller amt of pixels
        self.action_space = MultiBinary(12) # type of actions that can be taken
        self.health = 144
        self.enemy_health = 144
        self.score = 0
        self.matches_won = 0
        self.continue_timer = 100
        self.enemy_matches_won = 0
        self.previous_action = np.zeros(12)
        self.combo_scaler = 1
        self.last_damage_instance = 0
        # Startup and instance of the game 
        self.game = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis', use_restricted_actions=retro.Actions.FILTERED) # used to get valid button combos
    
    def reset(self): # restart
        # Return the first frame 
        obs = self.game.reset()
        obs = self.preprocess(obs) 
        self.previous_frame = obs # sets previous frame to current frame
        
        # Create a attribute to hold the score delta 
        self.score = 0 
        return obs
    
    def preprocess(self, observation): # grayscale, resize
        # Grayscaling 
        gray = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)
        # Resize 
        resize = cv2.resize(gray, (84,84), interpolation=cv2.INTER_CUBIC)
        # Add the channels value
        channels = np.reshape(resize, (84,84,1))
        return channels 
    
    def reward_function(self, state):
        # Extract variables
        continuetimer = state['continuetimer']
        enemy_matches_won = state['enemy_matches_won']
        enemy_health = state['enemy_health']
        health = state['health']
        matches_won = state['matches_won']
        score = state['score']

        # Initialize reward
        reward = 0

        # Reward for increasing score each frame (scaled down to avoid excessively large rewards)
        reward += score * 0.001  

        # Reward for inflicting damage on the enemy, only if enemy_health is reduced. Bonus for combos
        if enemy_health < self.enemy_health:
            if self.last_damage_instance < 200:
                self.combo_scaler *= 2
            else:
                self.combo_scaler = 1

            self.last_damage_instance = 0

            reward += (self.enemy_health - enemy_health) * 100 * self.combo_scaler

        # Penalty for losing health (scaled so health loss gives a clear penalty)
        if health < self.health:
            reward -= (self.health - health) * 100

        # Bonus if the player maintains health
        reward += health * 0.5  

        # Encourage the AI to win matches
        if matches_won > self.matches_won:
            reward += 2000  # Winning a match should give a significant reward

        # Penalize the AI if the enemy wins a match
        if enemy_matches_won > self.enemy_matches_won:
            reward -= 2000

        # Update previous states to enable frame-by-frame comparison
        self.enemy_health = enemy_health
        self.health = health
        self.matches_won = matches_won
        self.enemy_matches_won = enemy_matches_won
        self.continue_timer = continuetimer
        self.score = score
        self.last_damage_instance += 1

        return reward
    
    def step(self, action): # how do we process action
        # Take a step 
        obs, reward, done, info = self.game.step(action)
        obs = self.preprocess(obs) 

        self.previous_action = action
        
        # Frame delta 
        frame_delta = obs - self.previous_frame # change in pixels (was dropped in final model of tutorial)
        self.previous_frame = obs 
        
        # Reshape the reward function
        reward = self.reward_function(info)

        return frame_delta, reward, done, info
    
    def render(self, *args, **kwargs): # unpack any args and kwargs from stable baseline
        self.game.render()
        
    def close(self):
        self.game.close()

In [76]:
# Importing the optimzation frame - HPO
import optuna
# PPO algo for RL
from stable_baselines3 import PPO
# Bring in the eval policy method for metric calculation
from stable_baselines3.common.evaluation import evaluate_policy
# Import the sb3 monitor for logging 
from stable_baselines3.common.monitor import Monitor
# Import the vec wrappers to vectorize and frame stack
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
# Import os to deal with filepaths
import os

In [77]:
LOG_DIR = './logs/'
OPT_DIR = './opt/'

In [78]:
# Function to return test hyperparameters - define the object function
def optimize_ppo(trial): 
    return {
        'n_steps':trial.suggest_int('n_steps', 2048, 8192), # number of frames used in one batch of training (must use a factor of 64) (maybe take a number and multiply it by 64? 😎🤝😈)
        'gamma':trial.suggest_loguniform('gamma', 0.8, 0.9999), # discount rate
        'learning_rate':trial.suggest_loguniform('learning_rate', 1e-5, 1e-4), # how fast we tune optimizer (Critic and Actor for PPO)
        'clip_range':trial.suggest_uniform('clip_range', 0.1, 0.4), # how far we want to clip for our advantage value in PPO
        'gae_lambda':trial.suggest_uniform('gae_lambda', 0.8, 0.99) # smoothing parameter (used when calculating advantage)
    }

# IF U WANT TO USE OTHER ALGOS THE HYPERPARAMS MUST BE SWITCHED AS WELL (DQN, SAC, etc.)

In [79]:
SAVE_PATH = os.path.join(OPT_DIR, 'trial_{}_best_model'.format(1))

In [80]:
# Run a training loop and return mean reward 
def optimize_agent(trial):
    # try:
        model_params = optimize_ppo(trial)  # get dict of hyperparams

        # Create environment 
        env = StreetFighter()
        env = Monitor(env, LOG_DIR) # allowd for logging mean episode reward and mean episode length
        env = DummyVecEnv([lambda: env])
        env = VecFrameStack(env, 4, channels_order='last')

        # Create algo 
        model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=0, **model_params) # uses model params dict
        model.learn(total_timesteps=10000) # 100000 for production model (longer if can)

        # Evaluate model 
        mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=1) # 5 different games usually but went lower cuz i dont got cuda (those who know)
        env.close()

        SAVE_PATH = os.path.join(OPT_DIR, 'trial_{}_best_model'.format(trial.number))
        model.save(SAVE_PATH)

        return mean_reward

    # except Exception as e: # used in case hyperparam throws error (allows to continue training)
    #     return -1000

In [83]:
# Creating the experiment 
study = optuna.create_study(direction='maximize') # since mean reward is positive we maximize, otherwise minimize
study.optimize(optimize_agent, n_trials=1, n_jobs=1) # for prod used n_trials=100

[I 2024-11-04 21:32:35,836] A new study created in memory with name: no-name-7d1dc66d-ecb0-4aa7-900d-79738824c5fc
/var/folders/d4/_v_lx6gx66z8htqrwxr89lvc0000gn/T/ipykernel_4347/3706438210.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gamma':trial.suggest_loguniform('gamma', 0.8, 0.9999), # discount rate
/var/folders/d4/_v_lx6gx66z8htqrwxr89lvc0000gn/T/ipykernel_4347/3706438210.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate':trial.suggest_loguniform('learning_rate', 1e-5, 1e-4), # how fast we tune optimizer (Critic and Actor for PPO)
/var/folders/d4/_v_lx6gx66z8htqrwxr89lvc0000gn/T/ipykernel_4347/3706438210.py:7: FutureWarning: suggest_uniform

In [54]:
model = PPO.load(os.path.join(OPT_DIR, 'trial_0_best_model.zip'))

In [82]:
env.close()

In [56]:
# Import base callback 
from stable_baselines3.common.callbacks import BaseCallback

In [57]:
class TrainAndLoggingCallback(BaseCallback): # continuously learn by starting from best parameters done above

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [58]:
CHECKPOINT_DIR = './train/'

In [59]:
callback = TrainAndLoggingCallback(check_freq=1000, save_path=CHECKPOINT_DIR)

# Train Model

In [60]:
# Create environment 
env = StreetFighter()
env = Monitor(env, LOG_DIR)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')

In [61]:
model_params = study.best_params
model_params['n_steps'] = 7488  # set n_steps to 7488 or a factor of 64
# model_params['learning_rate'] = 5e-7 -> if really slow at training
model_params

{'n_steps': 7488,
 'gamma': 0.9275158942172772,
 'learning_rate': 5.27678066103925e-05,
 'clip_range': 0.27322085092983683,
 'gae_lambda': 0.9423129502987382}

In [62]:
model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, **model_params) # verbose 1 shows results as training

Using cpu device
Wrapping the env in a VecTransposeImage.


In [63]:
# Reload previous weights from HPO
model.load(os.path.join(OPT_DIR, 'trial_0_best_model.zip'))

In [64]:
# Kick off training 
model.learn(total_timesteps=5000, callback=callback) # timestep 5000000 recommended

Logging to ./logs/PPO_8


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 5.68e+03 |
|    ep_rew_mean     | 2.12e+05 |
| time/              |          |
|    fps             | 561      |
|    iterations      | 1        |
|    time_elapsed    | 13       |
|    total_timesteps | 7488     |
---------------------------------


In [45]:
# tensorboard --logdir=. 
# cd to logs
# ^ use to visually see learning progress

# Evaluate Model

In [31]:
model = PPO.load('./train/best_model_7000.zip')

In [32]:
# mean_reward, _ = evaluate_policy(model, env, render=True, n_eval_episodes=1)

# Testing Model

In [65]:
obs = env.reset()

In [66]:
# Reset game to starting state
obs = env.reset()
# Set flag to flase
done = False
for game in range(1): 
    while not done: 
        if done: 
            obs = env.reset()
        env.render()
        action = model.predict(obs)[0]
        obs, reward, done, info = env.step(action)
        time.sleep(0.01)
        print(reward)

[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.]
[88.

In [67]:
env.close()